<a href="https://www.kaggle.com/code/yaaangzhou/playground-s3-e21-very-simple-approach?scriptVersionId=141764821" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

Based on public kernal. thanks to all for your efforts.

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.linear_model import SGDOneClassSVM

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df = pd.read_csv('/kaggle/input/playground-series-s3e21/sample_submission.csv')

In [3]:
target = 'target'
num_var = df.columns.drop(['target','id'])

In [4]:
df = df[df.id != 2365]
df.target = df.target.clip(7, 20)

In [5]:
# SGD
clf = SGDOneClassSVM(nu=0.55).fit(df.drop(['id'],axis=1))
df.drop(np.array(np.where(clf.predict(df.drop(['id'],axis=1))==-1))[0], axis=0, inplace=True)
df = df.reset_index(drop=True)

# IsolationForest
lf = IsolationForest(random_state=0).fit(df.drop(['id'],axis=1))
df.drop(np.array(np.where(lf.predict(df.drop(['id'],axis=1))==-1))[0], axis=0, inplace=True)
df = df.reset_index(drop=True)

# LOF
lof = LocalOutlierFactor(n_neighbors=3, contamination=0.1)
predictions = lof.fit_predict(df[num_var])
df = df.drop(index=np.where(predictions == -1)[0])
df.reset_index(drop=True, inplace=True)

# Tricks from https://www.kaggle.com/code/warcoder/lb-1-32253-lof-svm-iforest-cleanlab
lowest_quality_labels = [2294,448, 437,309,1684,773,1722,2007]
df.drop(list(lowest_quality_labels),inplace=True)

In [6]:
drop_index = np.random.choice(df[df['target']==7].index.to_numpy(), 100, replace=False)
df.loc[drop_index]

,id,target,O2_1,O2_2,O2_3,O2_4,O2_5,O2_6,O2_7,NH4_1,...,NO3_5,NO3_6,NO3_7,BOD5_1,BOD5_2,BOD5_3,BOD5_4,BOD5_5,BOD5_6,BOD5_7
2736,3112,7.0,4.20,6.90,8.700,6.900,5.920,8.440,6.430,0.220,...,7.690,2.020,2.730,3.40,4.40,6.330,6.000,5.200,2.230,1.600
2398,2739,7.0,5.90,6.90,9.600,6.240,5.200,8.430,7.290,0.280,...,7.850,1.730,1.780,2.84,3.21,2.025,10.200,7.750,2.040,2.900
1317,1507,7.0,5.89,6.49,9.545,6.370,8.110,8.980,4.970,0.020,...,8.450,2.650,1.800,3.15,4.20,3.950,4.725,9.550,4.725,4.450
807,924,7.0,9.10,10.60,9.545,7.980,0.636,10.020,8.250,0.490,...,4.100,3.720,3.640,3.40,2.40,2.230,4.500,6.025,3.150,4.200
2927,3322,7.0,5.30,6.60,9.265,9.265,1.230,10.375,5.890,0.300,...,4.200,2.025,1.730,2.40,2.20,2.220,5.725,8.725,1.230,2.025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1093,1248,7.0,4.90,5.40,10.200,7.170,5.300,8.140,6.170,0.280,...,0.934,0.640,0.567,5.40,4.80,4.150,6.580,6.480,3.520,3.000
390,446,7.0,4.90,8.20,6.025,7.800,8.010,8.980,6.375,0.180,...,16.645,1.730,1.800,5.20,6.60,8.465,10.465,9.550,8.250,10.370
2482,2831,7.0,5.40,8.40,9.000,9.265,3.290,8.430,2.310,0.900,...,2.020,0.636,1.800,5.00,5.40,4.725,5.725,5.725,2.950,4.400
2574,2934,7.0,6.63,6.63,12.020,6.830,3.240,8.980,6.940,0.215,...,7.140,0.820,1.940,4.90,4.90,3.900,4.810,7.300,3.100,3.000


In [7]:
col_to_keep = ['O2_1', 'O2_2', 'BOD5_5', 'target']

for i in df.columns:
    if i not in col_to_keep:
        df[i]=0

In [8]:
df.to_csv('submission.csv',index=False)
df

,id,target,O2_1,O2_2,O2_3,O2_4,O2_5,O2_6,O2_7,NH4_1,...,NO3_5,NO3_6,NO3_7,BOD5_1,BOD5_2,BOD5_3,BOD5_4,BOD5_5,BOD5_6,BOD5_7
0,0,8.59,7.500,9.000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,16.645,0,0
1,0,9.10,13.533,40.900,0,0,0,0,0,0,...,0,0,0,0,0,0,0,5.725,0,0
2,0,8.21,3.710,5.420,0,0,0,0,0,0,...,0,0,0,0,0,0,0,6.750,0,0
3,0,8.39,8.700,8.100,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8.670,0,0
4,0,8.07,8.050,8.650,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8.400,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3072,0,8.65,8.367,8.760,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8.415,0,0
3073,0,8.08,6.250,8.300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,7.695,0,0
3074,0,8.09,6.630,6.630,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8.415,0,0
3075,0,9.95,8.367,8.433,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8.400,0,0
